# Import

In [1]:
!pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv
!pip install -qU langchain-core langchain-upstage langchain-chroma langchain langgraph langchain-community
!pip install -qU python-dotenv
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document
!pip install tavily-python
from langchain_community.retrievers import TavilySearchAPIRetriever

# API

In [2]:
import os
import getpass
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
from IPython import get_ipython

upstage_api_key_env_name = "UPSTAGE_API_KEY"
langchain_api_key_env_name = "LANGCHAIN_API_KEY"
tavily_api_key_env_name = "TAVILY_API_KEY"

def load_env():
    # Running in Google Colab
    if "google.colab" in str(get_ipython()):
        from google.colab import userdata
        upstage_api_key = userdata.get(upstage_api_key_env_name)
        langchain_api_key = userdata.get(langchain_api_key_env_name)
        tavily_api_key = userdata.get(tavily_api_key_env_name)
        return (os.environ.setdefault("UPSTAGE_API_KEY", upstage_api_key),
        os.environ.setdefault("LANGCHAIN_API_KEY", langchain_api_key),
        os.environ.setdefault("TAVILY_API_KEY", tavily_api_key))
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv
        load_dotenv()
        return (os.environ.get(upstage_api_key_env_name),
        os.environ.get(langchain_api_key_env_name),
        os.environ.get(tavily_api_key_env_name))

UPSTAGE_API_KEY, LANGCHAIN_API_KEY, TAVILY_API_KEY = load_env()

# Load DB

안 줄인 데이터베이스 사용!

In [11]:
persist_directory = r"C:\upstage\chroma_db"

# Set up the embedding function
embedding_function = UpstageEmbeddings(model="solar-embedding-1-large")

# Load the vector store from the persist directory
db = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)

retriever = db.as_retriever(search_kwargs={"k": 3})

# Chatbot

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_upstage import ChatUpstage

In [5]:
system_prompt = """
당신은 청년 정책 전문가입니다. 아래 지시 사항을 반드시 지켜서, 청년 정책에 관한 질문에 정확하고 오류 없는 답변을 생성하세요. \

<context 기반 답변 생성 방법>

1. context 기반 답변 : 다음 검색된 context를 기반으로, 질문에 대한 답변을 생성하세요. \
2. context가 없는 경우 : 답변 생성을 위한 context가 없는 경우, "잘 모르겠습니다."라고 답하세요. \
3. 내용의 일관성 유지 : 답변 생성 시, context의 내용을 있는 그대로 서술하고, context에 없는 내용을 절대 생성하지 마세요.\
4. 세부사항 일치 확인 : context의 세부적인 내용과 생성한 답변의 세부적인 내용은 동일해야 합니다. \
5. **금액, 날짜, 숫자 검증 : 특히, 생성한 답변에 금액, 날짜, 숫자가 포함된 경우, context와 동일한지 확인하고, 동일하지 않으면 답변을 삭제하세요.** \
6. 부정확한 내용 삭제 : 생성한 답변에서, **context와 다른 내용은 틀렸으니 삭제하세요. \**
7. 찾은 정책들이 비슷하여 하나로 특정하기 어려운 경우 사용자에게 list를 제공하거나 더 자세한 정보를 요구하세요.\

<질문 유형 기반 답변 생성 방법>

1. 추천 질문 : 정책 추천을 요구하는 질문이라면, 유저의 나이, 거주지, 취업 상태 등을 바탕으로 유저가 지원 가능한 정책을 찾고, 이에 대한 근거를 설명하세요. 이때, 유저의 거주지는 정책을 주관하는 지자체에 포함되어야 합니다. 예를 들어, 유저가 경기도에 거주하는 경우, 충청도가 주관하는 정책을 추천하면 안 되지만, 전국 단위로 주관하는 정책은 추천 가능합니다. \
2. 후기 관련 질문 : “후기”라는 단어가 포함된 질문이라면, 로컬 context가 아닌 tavily의 외부 검색 기능을 활용하세요. \
3. 세부 정보 질문 : 정책의 세부적인 정보를 묻는 질문이라면, 해당 정책명이 정확하게 포함된 context만을 검색해 답변을 생성하세요. \

<기타> 

1. 간결하고 이해하기 쉽게 답변하세요. \
2. 질문에 대한 집중 : 물어본 질문에만 답변하세요. \
3. 명확하지 않은 질문은 다시 : 질문을 명확하게 이해하지 못한 경우, “질문을 다시 구체적으로 해주세요”라고 답하세요. \
4. 질문이 여러 개의 정보를 요구하는 경우, 하위 질문으로 나누어 단계별로 생각하고 답변을 생성하세요.\

context: {context}
"""

In [12]:
''' <지시 + context + 대화 기록 + 질문> 템플릿. 이걸 llm에 입력으로 넣음 '''
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 유저 질문
    ]
)

llm = ChatUpstage(max_tokens=200, temperature=0.2, top_p=0.1)
chain = prompt | llm | StrOutputParser()

In [13]:
"""
Represents the state of our graph.
Attributes:
    context: retrieved context
    question: question asked by the user
    answer: generated answer to the question
    groundedness: groundedness of the assistant's response
    chat_history: list of chat messages
"""
from typing import TypedDict
from typing import List
class RagState(TypedDict):
    context: str
    question: str
    answer: str
    groundedness: str
    chat_history: List[str]

In [14]:
''' prepare RAG pipeline '''

''' 검색된 문서들의 내용을 하나의 문자열로 포맷 '''
def format_documents(docs: List[Document]) -> str:
    return "\n".join([doc.page_content for doc in docs if hasattr(doc, 'page_content')])


''' 검색 함수 '''
def retrieve(state: RagState) -> RagState:
    # retriever에서 질문과 관련된 문서들을 검색
    docs = retriever.invoke(state["question"])
    # 검색된 문서들의 내용을 하나의 문자열로 포맷 > context 생성
    context = format_documents(docs)
    # 생성된 context를 Ragstate의 context 변수에 저장
    return RagState(context = context)


''' 모델 답변 생성 함수 '''
def model_answer(state: RagState) -> RagState:
    # chain 실행 > 모델이 답변을 생성
    response = chain.invoke(state)
    # 생성된 response를 Ragstate의 answer 변수에 저장
    return RagState(answer = response)


''' 외부 검색 함수 '''
def tavily_search(state: RagState) -> RagState:
    tavily = TavilySearchAPIRetriever(k=3, include_raw_content=True, search_depth='advanced')
    print('Seaching using Tavily...')
    docs_external = tavily.invoke(state["question"])
    # tavily 검색 결과가 있으면
    if docs_external:
        context_tavily = format_documents(docs_external)
        # 외부 검색된 context를 Ragstate의 context 변수에 저장
        return RagState(context = context_tavily)
    else:
        print('No results found from Tavily.')
        return RagState(context = "검색 결과가 없습니다.")


from langchain_upstage import GroundednessCheck
gc = GroundednessCheck()

''' context와 answer를 비교하여 답변의 근거성 평가 '''
def groundedness_check(state: RagState) -> RagState:
    response = gc.run({"context": state["context"], "answer": state["answer"]})
    # 생성된 response를 Ragstate의 groundedness 변수에 저장
    return RagState(groundedness = response)

''' RagState의 groundedness 변수 값 출력 '''
def groundedness_condition(state: RagState) -> RagState:
    return state["groundedness"]

In [15]:
''' Build Graph '''
from langgraph.graph import END, StateGraph

workflow = StateGraph(RagState)
workflow.add_node("retrieve", retrieve)  
workflow.add_node("retrieve_again", retrieve) 

workflow.add_node("model", model_answer)  
workflow.add_node("model_again", model_answer)
workflow.add_node("model_tavily", model_answer)

workflow.add_node("groundedness_check_1", groundedness_check) 
workflow.add_node("groundedness_check_2", groundedness_check) 

workflow.add_node("tavily_search", tavily_search)  


# 상태간의 전이를 정의
workflow.add_edge("retrieve", "model")
workflow.add_edge("model", "groundedness_check_1")

workflow.add_edge("retrieve_again", "model_again")
workflow.add_edge("model_again", "groundedness_check_2")


# 근거성 평가 결과에 따라 다른 경로로 전이
workflow.add_conditional_edges(
    "groundedness_check_1",
    groundedness_condition,
    {
        "grounded": END,  # 근거 있음 > 워크플로우 종료
        "notGrounded": "retrieve_again",  # 근거 부족 > 다시 retrieve 단계로 전이
        "notSure": "retrieve_again",  # 불확실 > tavily_search 단계로 전이
    },
)

# 두 번째 근거성 평가 결과에 따라 tavily_search로 전이
workflow.add_conditional_edges(
    "groundedness_check_2",  # 두 번째 근거성 체크 후
    groundedness_condition,
    {
        "grounded": END,  # 근거 있음 > 워크플로우 종료
        "notGrounded": "tavily_search",  # 두 번째도 근거 부족 > tavily_search로 전이
        "notSure": "tavily_search",  # 두 번째도 불확실 > tavily_search로 전이
    },
)

# tavily_search 후에는 다시 모델로 이동
workflow.add_edge("tavily_search", "model_tavily")
workflow.add_edge("model_tavily", "groundedness_check_2")

# 워크플로우의 시작 지점을 검색 단계로 설정
workflow.set_entry_point("retrieve")

# 정의된 워크플로우를 컴파일하여 실행 가능한 상태로 만든다
app = workflow.compile()

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

keep_asking = True

while keep_asking:
    # 질문 입력 받기
    question = input()

    if question == 'ㅂ':
        print('대화를 종료합니다.')
        keep_asking = False
        break

    inputs = {
        "question": question,
        "chat_history": chat_history,
        "context": RagState["context"]
    }

    print(f'question: {question}')

    keys = []
    values = []

    answer = ""
    for output in app.stream(inputs):
        for key, value in output.items():
            keys.append(key)
            values.append(value)
            print(f"Node '{key}':{value}")
            print("\n---\n")

    answer = values[-2]['answer']  # gc 결과가 grounded가 나오기 직전의 답변을 저장

    chat_history += [str(HumanMessage(inputs["question"])), AIMessage(answer)]

question:  자립준비청년(청년 유형) 전세임대 정책의 신청기간은?
Node 'retrieve':{'context': '2024년 자립준비청년(청년 유형) 전세임대 입주자 수시모집• 자립준비청년(청년 유형) 전세임대란?- 입주대상자로 선정된 자립준비청년이 거주할 주택을 물색하면 LH에서 주택소유자와 전세계약을체결한 후 재임대하는 제도입니다.- 자립준비청년을 위한 전세임대제도는 청년 전세임대와 소년소녀 전세임대 두가지가 있으며, 본 공고는청년 전세임대 1순위에 해당하는 자립준비청년을 위한 공고입니다. (소년소녀가정용 전세임대 지원을위한 자립준비청년의 입주신청은 주민등록지 행정복지센터로 하시면 됩니다.)- 본 입주자 모집의 신청내용 수정은 신청일 24:00(마감일의 경우 18:00)까지 가능하고, 이후변경내용에 대해서는 관할 지역본부로 별도문의 바랍니다.1. 사업대상지역 : 전국2. 신청방법■ 청년 전세임대 청약은 인터넷으로만 신청 가능LH청약플러스(https://apply.lh.or.kr)→임대주택(청약신청)<유의사항>· 인터넷 청약시스템은 인증서(공동인증서, 금융인증서 또는 네이버 인증서 등)를 소지한 경우에 한하여 신청가능하며, LH 홈페이지에서 제공하는 “청약신청 연습하기”를 통하여 사전에 미리 청약 절차를 연습하여청약하시기 바랍니다.3. 신청자격■신청자격 : 신청일 현재 본인이 무주택자이면서 「아동복지법」 제16조 및 제16조의3에 따라 가정위탁 보호조치가종료되거나 아동복지시설에서 퇴소한지 5년 이내인 사람(보호조치를 연장한 자, 보호조치 종료예정자, 시설 퇴소예정자 포함, 혼인중인 자는 제외)4. 신청절차, 기간 및 방법■ 신청절차■ 신청기간 : 2024. 01. 02.(화) 10:00 ~ 2024. 12. 31.(화) 18:00■ 신청방법 : 제출서류 스캔 후 첨부하여 LH 청약센터(https://apply.lh.or.kr)에서 신청- 제한된 첨부파일의 용량 · 파일형식 확인, 스캔 등 오류로 인해 서류 식별이 불가하지 않도록 주의(파일용량 최대

# Gradio

In [18]:
!pip install -qU gradio
import gradio as gr

In [20]:
''' gradio에서 대화 진행 '''

chat_history = []

# Gradio에서 처리될 대화 함수
def chat(question, context):
    global chat_history

    inputs = {
        "question": question,
        "chat_history": chat_history,
        "context": context
    }

    keys = []
    values = []

    answer = ""
    for output in app.stream(inputs):
        for key, value in output.items():
            keys.append(key)
            values.append(value)
    
    answer = values[-2]['answer']  # gc 결과가 grounded가 나오기 직전의 답변을 저장

    # 대화 기록 업데이트
    chat_history += [str(HumanMessage(inputs["question"])), AIMessage(answer)]

    # Gradio에서 UI로 반환할 값
    return f"{answer}"

with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "저는 경기도에 거주하는 23살 대학생입니다. 저에게 맞는 정책을 추천해주세요!"
        ],
        title="청년 정책 추천 챗봇",
        description="당신께 꼭 필요한 청년 정책을 추천해드릴게요!",
    )
    chatbot.chatbot.height = 500

# Gradio 실행
demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4fce3274bf8dfd58c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4fce3274bf8dfd58c6.gradio.live
